# MLB Player Unique Identifier Equivalencies

## Introduction

There are many different resources that keep records of MLB data. When trying to combine and compare data from different resources, sometimes it can be difficult depending on the way that the data is formatted.

I ran into this problem while digging into some MLB data. In my analysis, I wanted to use two different sources that both have entries for every single MLB player. The problem was that there was no simple way to cross-compare all players between the two data sources.

### Data Sources

The two data sources I wanted to use were [Baseball Reference](https://www.baseball-reference.com/) and [RetroSheet](https://www.retrosheet.org). 

Most players (about 90%) can be matched between the two sources based on their name alone. This leaves a substantial subset (10%) of players that require more careful inspection. 

Thankfully, both of these sources use "unique identifiers" to identify each individual player. 

#### Unique Identifiers

Unique identitifers are a special kind of ID. In this context, a unique identifier is a string of letter and numbers guaranteed to match up to exactly one player. Two players would never share the same unique identifier (even if they share the same exactly name) and a single player would never be assigned two different unique identifiers (by the same source).

Baseball Reference and RetroSheet use similar formatting for player unique identifiers, but they do not match exactly. For example, LaMonte Wade Jr.'s unique identifier in Baseball Reference is `wadela01` and in RetroSheet is `wadel001`.

If I had a table that matched up Baseball Reference's unique identifiers to RetroSheet's unique identifiers, I would be able to easily and smoothly compare player data across the two sourcees

### Main Objective

For the final result, I would like to have a table with a single row for every 2022 MLB player, and columns for the player's Name, Baseball Reference ID, and RetroSheet ID.

This Jupyter Python notebook demonstrates the process I used to create a table that matches player IDs from Baseball Reference with player IDs from RetroSheet. I include all the Python code I used, as well as an English description of my thought process.

This notebook specifically focuses on all 2022 MLB players, but this process could be edited and expanded to encompass more seasons.

The formatting will look as below:

In [1]:
import pandas as pd

In [2]:
exampleData = {'Name':['LaMonte Wade Jr.', 'First Last 1', 'First Last 2', '...'],
               'Baseball Reference ID':['wadela01', 'lastfi01', 'lastfi02', '...'],
               'RetroSheet ID':['wadel001', 'lastf001', 'lastf002', '...']}
resultTable = pd.DataFrame(exampleData)

resultTable

,Name,Baseball Reference ID,RetroSheet ID
0,LaMonte Wade Jr.,wadela01,wadel001
1,First Last 1,lastfi01,lastf001
2,First Last 2,lastfi02,lastf002
3,...,...,...


## Data Collection

I collected data from [Baseball Reference](https://www.baseball-reference.com/) and [RetroSheet](https://www.retrosheet.org). Since these are active websites, it's possible they may change or go offline in the future. This data collection process is up-to-date as of January 2023. 

### Baseball Reference

From Baseball Reference, I used the these two tables:
* [2022 Player Standard Pitching](https://www.baseball-reference.com/leagues/majors/2022-standard-pitching.shtml) table
* [2022 Player Standard Batting](https://www.baseball-reference.com/leagues/majors/2022-standard-batting.shtml) table

To save this data, I used the "Share & Export" menu ⇨ "Get Table as CSV (for Excel)"

Then I copied the plain CSV text and pasted it into a simple text editor (I used TextEdit) and saved the files as `BaseballReference_StandardPitching_2022.csv` and `BaseballReference_StandardBatting_2022.csv` For both files, I omitted the last line of the CSV produced by Baseball Reference that describes Leauge Averages since it is completely irrelevant to this project.

I put these two files in a folder called `Baseball Reference`, which I have in the same folder/directory as this Jupyter notebook.

*This is important for running the code in this notebook, if the files are stored anywhere else then this will result in errors or require code edits.*

In [3]:
bbref_filename_list = ['BaseballReference_StandardPitching_2022.csv',
                       'BaseballReference_StandardBatting_2022.csv']

bbref_folder = 'Baseball Reference'

### RetroSheet

From RetroSheet, I used the 30-file collection of roster files from the "2022 Play-by-Play, Regular Season Event Files" download.

To access this download, I used the "Data downloads" menu ⇨ "Play-by-play files"

Then, under "Regular Season Event Files," I cliked the "2022" link and a file called `2022eve.zip` automatically downloaded. In my downloads folder, I double-clicked the file and it unzipped into a folder called `2022eve`. I put this folder into a folder called `RetroSheet`. 

*This is important for running the code in this notebook, if the files are stored anywhere else then this will result in errors or require code edits.*

In [4]:
retrosheet_folder = 'RetroSheet/2022eve'

## Data Exploration

### Baseball Reference

Both the Player Standard Pitching table and the Player Standard Batting table list every MLB player who pitched or batted (respectively) in 2022.

Some players are represented by multiple rows. This occurrs when a player appeared for multiple teams in the season.

Each player's name, age, team and leauge are listed. There are also many counting, rate, and advanced statistics listed. The last column, called `Name-additional`, is the player's Baseball Reference unique identifier. In this project, I'll be focusing on just the players' names and unique identifiers.

Left handed pitchers and batters are denoted with an asterisk (`*`) after their name. Switch hitters are denoted with a pound sign (`#`).

With respect to player names, I notice two important things. The first is that player names include character accents. The second is that Baseball Reference uses a unicode character called `No-Break Space` (` `) in between components of player names instead of the standard `Space` (` `) character.

### RetroSheet

The 2022 "Regular Season Event Files" download contains 61 files:
* 1 file called `TEAM2022`
* 30 "Event Files" ending in `.EVN` or `.EVA`
* 30 roster files ending in `.ROS`

In this project, I need the 30 roster files. There is one roster file for each MLB team, listing all the MLB players for that team in 2022. Each line corresponds to one player, and includes the following infomation:
* RetroSheet unique identifier
* Last name
* First name
* Bating handedness
* Throwing handedness
* Team
* Position

Even though RetroSheet does not choose to use the `.csv` extension, these roster files are still essentially comma separated value files.

In this project, I'll be focusing on just the players' names and unique identifiers.

[RetroSheet offical documentation is available on their website.](https://www.retrosheet.org/eventfile.htm)

## Data Preparation

In order to process the player name and unique ID information, I need to do some pre-processing to prepare the data.

The final result of this step will be two tables, one for each data source. Each table will have a row for each 2022 MLB player, with a column for their name and a column for their respective ID.

### Baseball Reference

First, I'll import the two Baseball Reference files and combine them into one table. Then I'll do some processing to remove duplicate rows and adjust the formatting of the player names.

In [5]:
# Import both the Batting and Pitching tables from the .csv files
# Using only the Name and Name-additional (unique ID) columns
bbref_pitching_path = bbref_folder + '/' + bbref_filename_list[0]
bbref_pitching_dataframe = pd.read_csv(bbref_pitching_path, usecols = ['Name', 'Name-additional'])

bbref_batting_path = bbref_folder + '/' + bbref_filename_list[1]
bbref_batting_dataframe = pd.read_csv(bbref_batting_path, usecols = ['Name', 'Name-additional'])

In [6]:
# Combine/concatenate Batting and Pitching Dataframes into one Dataframe
bbref_playerID_dataframe = pd.concat([bbref_batting_dataframe, bbref_pitching_dataframe])

In [7]:
# Rename 'Name-additional' column to 'Baseball Reference ID'
bbref_playerID_dataframe = bbref_playerID_dataframe.rename(columns = {'Name-additional' : 'Baseball Reference ID'})
bbref_playerID_dataframe

,Name,Baseball Reference ID
0,CJ Abrams*,abramcj01
1,CJ Abrams*,abramcj01
2,CJ Abrams*,abramcj01
3,Albert Abreu,abreual01
4,Albert Abreu,abreual01
...,...,...
1063,Rob Zastryzny*,zastrro01
1064,Rob Zastryzny*,zastrro01
1065,Angel Zerpa*,zerpaan01
1066,T.J. Zeuch,zeuchtj01


Notice that there are many duplicated rows. This is because Baseball Reference includes separate rows for players that played on multiple teams in the season. This can be useful when investigating statistics or other infomation.

Since I'm just focusing on player names and unique IDs, I can safely remove duplicate rows. I'll also remove the asterisk (` `) and pound sign (`#`) symbols, and change the `No-Break Space` (` `) to the `Space` (` `) character.

In [8]:
# Remove rows with duplicated IDs
duplicatedID_status_list = bbref_playerID_dataframe.duplicated('Baseball Reference ID')
bbref_playerID_dataframe = bbref_playerID_dataframe[ ~duplicatedID_status_list]

In [9]:
# Remove * and #
bbref_playerID_dataframe = bbref_playerID_dataframe.replace(to_replace = '\*|#',
                                                            value = '', regex = True)
# Replace "No-Break Space" with "Space"
nobreakspace_char = ' '
space_char = ' '
bbref_playerID_dataframe = bbref_playerID_dataframe.replace(to_replace= nobreakspace_char,
                                                            value = space_char, regex = True)

In [10]:
# Sort by name column and reset the Dataframe indices
bbref_playerID_dataframe = bbref_playerID_dataframe.sort_values('Name').reset_index(drop = True)

In [11]:
bbref_playerID_dataframe

,Name,Baseball Reference ID
0,A.J. Alexy,alexyaj01
1,A.J. Ladwig,ladwiaj01
2,A.J. Minter,minteaj01
3,A.J. Puk,pukaj01
4,AJ Pollock,polloaj01
...,...,...
1490,Zack Littell,litteza01
1491,Zack Short,shortza01
1492,Zack Thompson,thompza02
1493,Zack Weiss,weissza01


### RetroSheet

First, I'll get all the roster files from the RetroSheet Event Files folder and read all the lines/entries into a list. Then I'll make a table from this data, and isolate the player names and RetroSheet IDs.

In [12]:
import os

In [13]:
# Detemine the current directory and combine with RetroSheet folder directory
current_directory = os.getcwd()
retrosheet_directory = '/' + retrosheet_folder

retrosheet_path = current_directory + retrosheet_directory

In [14]:
# Loop through all the files in the RetroSheet Directory and add them to a list
roster_filename_list = []
for filename in os.listdir(path = retrosheet_path):
    
    if filename.find('.') != -1:
        if filename.split('.')[1] == 'ROS':
            roster_filename_list.append(filename)

In [15]:
# Loop through all the .ROS roster files and add each line to a list
roster_entries_list = []
for roster_filename in roster_filename_list:
    
    roster_path = retrosheet_folder + '/' + roster_filename
    
    with open(roster_path, 'r') as file:
        roster_entries = file.readlines()
        
        for entry in roster_entries:
            roster_entries_list.append(entry.rstrip())

In [16]:
# Create a DataFrame from the list of roster entries
retrosheet_playerID_dataframe = pd.DataFrame([entry.split(',') for entry in roster_entries_list],
                                        columns = ['RetroSheet ID', 'Last', 'First',
                                                   'Bats', 'Throws', 'Team', 'Position'])

In [17]:
# Make a new column called "Name" by combing the first and last name columns
retrosheet_playerID_dataframe['Name'] = retrosheet_playerID_dataframe['First'] + ' ' + retrosheet_playerID_dataframe['Last']

# Drop the columns we don't care about for this project
retrosheet_playerID_dataframe = retrosheet_playerID_dataframe.drop(columns = ["Last", "First", "Bats", "Throws",
                                                                    "Team", "Position"])

In [19]:
# Remove names with duplicated IDs
duplicatedID_status_list = retrosheet_playerID_dataframe.duplicated('RetroSheet ID')
retrosheet_playerID_dataframe = retrosheet_playerID_dataframe[ ~duplicatedID_status_list]

In [21]:
# Sort by name and reset the Dataframe indices
retrosheet_playerID_dataframe = retrosheet_playerID_dataframe.sort_values('Name').reset_index(drop = True)

### Data Preparation Results

In [23]:
bbref_playerID_dataframe

,Name,Baseball Reference ID
0,A.J. Alexy,alexyaj01
1,A.J. Ladwig,ladwiaj01
2,A.J. Minter,minteaj01
3,A.J. Puk,pukaj01
4,AJ Pollock,polloaj01
...,...,...
1490,Zack Littell,litteza01
1491,Zack Short,shortza01
1492,Zack Thompson,thompza02
1493,Zack Weiss,weissza01


In [24]:
retrosheet_playerID_dataframe

,RetroSheet ID,Name
0,alexa001,A. J. Alexy
1,ladwa001,A. J. Ladwig
2,minta001,A. J. Minter
3,puk-a001,A. J. Puk
4,polla001,AJ Pollock
...,...,...
1490,littz001,Zack Littell
1491,shorz001,Zack Short
1492,thomz002,Zack Thompson
1493,weisz001,Zack Weiss


## Determining ID Equivalencies
---

Now that we have our two tables, we can start the main process of creating our combined table!

There are different ways to approach this process, but I'll choose to use a combination of automated and manual processing. Since I'm only looking at the 2022 MLB players, and only about 10% (< 200) of them fall into the manual processing category, this is managable.

If this project were expanded to encompass larger sets of players, it may not be so managable to do manual editing with the approach I use here. In that situation, it could make sense to expand the scope of the coding work for automation to reduce or potentionally eliminate the manual editing portion. Given the non-standardized nature of human naming conventions, it's possible that some human attention will always be required.

I'll start by splitting up all the players into separate disjoint cases. Breaking down the data this way makes it much easier to work with.

* Player shares a name with at least one other player (`Case 1`)
* Player does not share a name with another player
    * Player name is listed identically between the two sources (`Case 2`)
    * Player name is listed differently between the two sources (`Case 3`)

`Case 2` is the case into which 90% of MLB players fall. This is the easist case to handle and does not require any manual processing.

`Case 1` and `Case 3` are the situations that require greater scrutiny. 

I will make three separate equivalency tables, one for each disjoint case. At the end, I will combine all three tables into one table to produce the final result.

### `Case 1`: Duplicate Names

First, I need to address the duplicate names. Some players share the same exact name, dealing with these cases first to makes the rest of the process much smoother.

To begin, I need to determine which names are duplicated. I'll do this separately for the two sources. Since Baseball Reference formats player names differently than RetroSheet, the results may be slightly different.

I'll use code to create these two tables, and then export the tables to `.csv` files and complete the manual editing in a program like Excel/Numbers/Google Sheets/etc. I'll export that result to another `.csv` file that I'll import back into this notebook as the final result for this case.

The last step is to remove this set of players from my working tables (`bbref_playerID_dataframe`) and (`retrosheet_playerID_dataframe`) so that only `Case 2` and `Case 3` players remain.

In [31]:
# Create a list of all the duplicated names in Baseball Reference
duplicatename_status = bbref_playerID_dataframe.duplicated('Name')
bbref_duplicatename_list = bbref_playerID_dataframe[duplicatename_status]['Name']

# Create a list of all the duplicated names in RetroSheet
duplicatename_status = retrosheet_playerID_dataframe.duplicated('Name')
retrosheet_duplicatename_list = retrosheet_playerID_dataframe[duplicatename_status]['Name']

In [32]:
# Create a DataFrame of the duplicated names in Baseball Reference
duplicatename_status = bbref_playerID_dataframe['Name'].isin(bbref_duplicatename_list)
bbref_duplicatename_dataframe = bbref_playerID_dataframe[duplicatename_status]

bbref_duplicatename_dataframe

,Name,Baseball Reference ID
385,Diego Castillo,castidi02
386,Diego Castillo,castidi01
899,Luis Castillo,castilu03
900,Luis Castillo,castilu02
904,Luis García,garcilu04
905,Luis García,garcilu03
910,Luis Ortiz,ortizlu03
911,Luis Ortiz,ortizlu02
1430,Will Smith,smithwi05
1431,Will Smith,smithwi04


In [33]:
# Create a DataFrame of the duplicated names in RetroSheet
duplicatename_status = retrosheet_playerID_dataframe['Name'].isin(retrosheet_duplicatename_list)
retrosheet_duplicatename_dataframe = retrosheet_playerID_dataframe[duplicatename_status]

retrosheet_duplicatename_dataframe

,RetroSheet ID,Name
385,castd004,Diego Castillo
386,castd002,Diego Castillo
899,castl004,Luis Castillo
900,castl003,Luis Castillo
903,garcl006,Luis Garcia
904,garcl007,Luis Garcia
905,garcl005,Luis Garcia
910,ortil002,Luis Ortiz
911,ortil003,Luis Ortiz
1430,smitw002,Will Smith


Notice that the RetroSheet table has 11 rows, while the Baseball Reference table has only 10 rows. This is an example of how player names are formatted differently between the two data sources. RetroSheet doesn't include any character accents.

RetroSheet lists `Luis Garcia` three times, while Baseball Reference lists `Luis García` twice. This means that Baseball Reference data should contain one row representing a player named `Luis Garcia` (with no i-acute character)

I want to make sure that this is accurate, and also lookup this player's Baseball Reference ID since I will include all 11 players in the final table for this case.

In [40]:
# Lookup 'Luis Garcia' in the Baseball Reference data
bbref_playerID_dataframe[ bbref_playerID_dataframe['Name'] == 'Luis Garcia']

,Name,Baseball Reference ID
903,Luis Garcia,garcilu05


In [41]:
# Create a new table, which will eventually contain the final result for this case
duplicatename_dataframe = pd.DataFrame(columns = ['Name',
                                                  'Baseball Reference ID',
                                                  'RetroSheet ID'])

In [42]:
# Copy the duplicated Baseball Reference player names and IDs into this table
duplicatename_dataframe['Name'] = bbref_duplicatename_dataframe['Name']
duplicatename_dataframe['Baseball Reference ID'] = bbref_duplicatename_dataframe['Baseball Reference ID']

This new table now has 10 rows, matching the Baseball Reference duplicated names table.

In [43]:
# Manually copy the data for 'Luis Garcia'
duplicatename_dataframe = duplicatename_dataframe.append({'Name' : 'Luis Garcia',
                                                         'Baseball Reference ID' :' garcilu05'},
                                                         ignore_index=True)

#### Manually Edited Portion

I'll export both of these tables to `.csv` files to open in a spreadsheet program like Excel. 

In [47]:
# Export final table (that needs to be filled in) to .csv file
filename = '2022_NeedsManualEdit_DuplicateName_FullTable.csv'
duplicatename_dataframe.to_csv(filename, index = False)

# Export table with RetroSheet ID information to .csv file
filename = '2022_NeedsManualEdit_DuplicateName_RetrosheetIDs.csv'
retrosheet_duplicatename_dataframe.to_csv(filename, index = False)

Manually editing these players' ID equivalencies requires some outside information. It's not possible to determine which IDs match up without using other information (for example, which team(s) the player played for). This information is accessible online and in the greater data collected for this project.

For the 2022 season, I was able to match all 11 players' IDs up using players' teams only. The process I used was first to open both tables in Apple Numbers. Then, starting with `Diego Castillo` / `castidi02`, I looked up each Baseball Reference ID on the Baseball Reference website. I checked which team this player played for in 2022, and then opened the corresponding RetroSheet roster file and found the RetroSheet ID that corresponded to this name.

This process works for all 2022 duplicate player names. If this project were to be expanded to a larger set of players, it's possible that this process would require more information to cross-reference and determine which IDs match.

After all the IDs were matched, I exported the table containing three columns (Name, Baseball Reference ID, RetroSheet ID) to a `.csv` file called `2022_ManualEdit_DuplicateName_FinalTable.csv`

Now I'll import that table back into this notebook.

In [48]:
filename = '2022_ManualEdit_DuplicateName_FinalTable.csv'
duplicatename_dataframe = pd.read_csv(filename)
duplicatename_dataframe

,Name,Baseball Reference ID,RetroSheet ID
0,Diego Castillo,castidi01,castd002
1,Diego Castillo,castidi02,castd004
2,Luis Castillo,castilu02,castl003
3,Luis Castillo,castilu03,castl004
4,Luis García,garcilu04,garcl006
5,Luis García,garcilu03,garcl005
6,Luis Ortiz,ortizlu02,ortil002
7,Luis Ortiz,ortizlu03,ortil003
8,Will Smith,smithwi05,smitw003
9,Will Smith,smithwi04,smitw002


Finally, I'll remove these players from the working tables `bbref_playerID_dataframe` and `retrosheet_playerID_dataframe` to prepare for the next case.

In [55]:
# Remove Case 1 players from Baseball Reference table
duplicatedID_list = duplicatename_dataframe['Baseball Reference ID']
duplicatedID_status_list = bbref_playerID_dataframe['Baseball Reference ID'].isin(duplicatedID_list)
bbref_playerID_dataframe = bbref_playerID_dataframe[ ~duplicatedID_status_list]

In [56]:
# Remove Case 1 players from RetroSheet table
duplicatedID_list = duplicatename_dataframe['RetroSheet ID']
duplicatedID_status_list = retrosheet_playerID_dataframe['RetroSheet ID'].isin(duplicatedID_list)
retrosheet_playerID_dataframe = retrosheet_playerID_dataframe[ ~duplicatedID_status_list]

### `Case 2`: Matching Names

This is the category into which 90% of players fall, where their name is listed the same way in both sets of data.

I'll use the built in merge function, which is similar to a SQL join. The functionality is not identical, this is discussed further in the [pandas documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.merge.html)

Similar to the previous case, I'll make a new table with the matching names and then remove them from the working tables.

In [60]:
# Use merge function to match up IDs
matchingname_dataframe = bbref_playerID_dataframe.merge(retrosheet_playerID_dataframe)
matchingname_dataframe

,Name,Baseball Reference ID,RetroSheet ID
0,AJ Pollock,polloaj01,polla001
1,Aaron Ashby,ashbyaa01,ashba003
2,Aaron Brooks,brookaa01,brooa001
3,Aaron Bummer,bummeaa01,bumma001
4,Aaron Civale,civalaa01,civaa001
...,...,...,...
1311,Zack Littell,litteza01,littz001
1312,Zack Short,shortza01,shorz001
1313,Zack Thompson,thompza02,thomz002
1314,Zack Weiss,weissza01,weisz001


In [61]:
# Remove these rows from the Baseball Reference and RetroSheet DataFrames
bbref_playerID_dataframe = bbref_playerID_dataframe[ ~bbref_playerID_dataframe['Baseball Reference ID'].isin(matchingname_dataframe['Baseball Reference ID'])]
retrosheet_playerID_dataframe = retrosheet_playerID_dataframe[ ~retrosheet_playerID_dataframe['RetroSheet ID'].isin(matchingname_dataframe['RetroSheet ID'])]

### `Case 3`: Non-Matching Names across data sources

This leaves all the names that don't match across the two data sources. 

Similar to `Case 1`, I'll export the two tables to `.csv` files and complete the manual editing in a spreadsheet program like Excel. I'll export that result to another `.csv` file, which I'll import back into this notebook as the final result for this case.

In [63]:
# Create a new table, which will eventually contain the final result for this case
nonmatchingname_dataframe = pd.DataFrame(columns = ['Name',
                                                   'Baseball Reference ID',
                                                   'RetroSheet ID'])
nonmatchingname_dataframe

,Name,Baseball Reference ID,RetroSheet ID


In [65]:
# Copy the remaining Baseball Reference player names and IDs into this table
nonmatchingname_dataframe['Name'] = bbref_playerID_dataframe['Name']
nonmatchingname_dataframe['Baseball Reference ID'] = bbref_playerID_dataframe['Baseball Reference ID']

In [67]:
retrosheet_playerID_dataframe

,RetroSheet ID,Name
0,alexa001,A. J. Alexy
1,ladwa001,A. J. Ladwig
2,minta001,A. J. Minter
3,puk-a001,A. J. Puk
18,mondr003,Adalberto Mondesi
...,...,...
1447,herny002,Yadiel Hernandez
1452,diazy001,Yandy Diaz
1456,mercy001,Yermin Mercedes
1460,lopey001,Yoan Lopez


In [68]:
# Export final table (that needs to be filled in) to .csv file
nonmatchingname_dataframe.to_csv('2022_NeedsManualEdit_NonMatchingName_FullTable.csv', index = False)

# Export table with RetroSheet ID information to .csv file
retrosheet_playerID_dataframe.to_csv('2022_NeedsManualEdit_NonMatchingName_RetroSheetIDs.csv', index = False)

#### Manually Edited Portion

I opened both tables in the spreadsheet program (I used Apple Numbers), and organized them so that they were next to each other. Then, I went down the sequentially one-by-one and copied the proper RetroSheet IDs into the full table. Since the tables are sorted alphabetically, most of the players (about 85%) are organized such that they occur in the same row in both tables.

This portion of manual editing requires great attention to detail. While most players do appear in the same row, there are blocks of rows where the order of players are mixed up.

If this project were to be expanded to process more than one season, it is this sub-process that would benefit most from more automation. I determined that for the scope of this project, since this is a one-time manual edit and there are only 168 players, it would not be the best investment of resources to invest in automating this, at this time.


In [69]:
nonmatchingname_dataframe = pd.read_csv('2022_ManualEdit_NonMatchingName_FinalTable.csv')

In [70]:
nonmatchingname_dataframe

,Name,Baseball Reference ID,RetroSheet ID
0,A.J. Alexy,alexyaj01,alexa001
1,A.J. Ladwig,ladwiaj01,ladwa001
2,A.J. Minter,minteaj01,minta001
3,A.J. Puk,pukaj01,puk-a001
4,Adalberto Mondesí,mondera02,mondr003
...,...,...,...
163,Yadiel Hernández,hernaya01,herny002
164,Yandy Díaz,diazya01,diazy001
165,Yermín Mercedes,merceye01,mercy001
166,Yoan López,lopezyo01,lopey001


### Combining All ID Equivalency Tables

Now that I have all 3 tables, I can complete the final step of combining them into one table.

In [72]:
duplicatename_dataframe

,Name,Baseball Reference ID,RetroSheet ID
0,Diego Castillo,castidi01,castd002
1,Diego Castillo,castidi02,castd004
2,Luis Castillo,castilu02,castl003
3,Luis Castillo,castilu03,castl004
4,Luis García,garcilu04,garcl006
5,Luis García,garcilu03,garcl005
6,Luis Ortiz,ortizlu02,ortil002
7,Luis Ortiz,ortizlu03,ortil003
8,Will Smith,smithwi05,smitw003
9,Will Smith,smithwi04,smitw002


In [71]:
matchingname_dataframe

,Name,Baseball Reference ID,RetroSheet ID
0,AJ Pollock,polloaj01,polla001
1,Aaron Ashby,ashbyaa01,ashba003
2,Aaron Brooks,brookaa01,brooa001
3,Aaron Bummer,bummeaa01,bumma001
4,Aaron Civale,civalaa01,civaa001
...,...,...,...
1311,Zack Littell,litteza01,littz001
1312,Zack Short,shortza01,shorz001
1313,Zack Thompson,thompza02,thomz002
1314,Zack Weiss,weissza01,weisz001


In [73]:
nonmatchingname_dataframe

,Name,Baseball Reference ID,RetroSheet ID
0,A.J. Alexy,alexyaj01,alexa001
1,A.J. Ladwig,ladwiaj01,ladwa001
2,A.J. Minter,minteaj01,minta001
3,A.J. Puk,pukaj01,puk-a001
4,Adalberto Mondesí,mondera02,mondr003
...,...,...,...
163,Yadiel Hernández,hernaya01,herny002
164,Yandy Díaz,diazya01,diazy001
165,Yermín Mercedes,merceye01,mercy001
166,Yoan López,lopezyo01,lopey001


In [75]:
finalresult_dataframe = pd.concat([duplicatename_dataframe,
                                   matchingname_dataframe,
                                   nonmatchingname_dataframe])

In [76]:
# Sort the table by name and reset the index values
finalresult_dataframe = finalresult_dataframe.sort_values('Name').reset_index(drop = True)
finalresult_dataframe

,Name,Baseball Reference ID,RetroSheet ID
0,A.J. Alexy,alexyaj01,alexa001
1,A.J. Ladwig,ladwiaj01,ladwa001
2,A.J. Minter,minteaj01,minta001
3,A.J. Puk,pukaj01,puk-a001
4,AJ Pollock,polloaj01,polla001
...,...,...,...
1490,Zack Littell,litteza01,littz001
1491,Zack Short,shortza01,shorz001
1492,Zack Thompson,thompza02,thomz002
1493,Zack Weiss,weissza01,weisz001


In [77]:
# Export this table to a .csv file
finalresult_dataframe.to_csv('2022_MLBUniqueIdentiferEquivalencies_BaseballReferenceRetroSheet.csv', 
                             index = False)